In [1]:
import json
import os
import numpy as np
from keras.models import Model,load_model
from keras.layers import Input, LSTM, Dense,Activation,Embedding,merge
from keras.layers.core import Lambda
from keras.layers import concatenate,Reshape,Concatenate,RepeatVector
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
from nltk.translate.bleu_score import sentence_bleu
from keras.callbacks import EarlyStopping
import random
from PIL import Image
from matplotlib import pyplot as plt
import pickle
from keras import backend as K
import math
import random

Using TensorFlow backend.


In [2]:
coco_json='./data/dataset_coco.json'
tokenfile='./tokenv2.pkl'
detokenfile='./detokenv2.pkl'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
KTF.set_session(session)

In [3]:
with open(coco_json) as file:
    lines=file.readlines()
data=json.loads(lines[0])['images']

In [4]:
train_x_file=[]
train_y_seq=[]
vali_x_file=[]
vali_y_seq=[]
token={}
if os.path.exists(tokenfile):
    with open(tokenfile,'rb') as file:
        token=pickle.load(file)
else:
    token['\t']=1
    token['\n']=2

In [5]:
for d in data:
    if d['split']=='train':
        train_x_file.append('./coco/'+d['filepath']+'/'+d['filename'])
        seq=d['sentences'][0]['tokens']
        for s in seq:
            if s not in token:
                token[s]=len(token)+1
        train_y_seq.append(['\t']+seq+['\n'])
    if d['split']=='test':
        vali_x_file.append('./coco/'+d['filepath']+'/'+d['filename'])
        seq=d['sentences'][0]['tokens']
        for s in seq:
            if s not in token:
                token[s]=len(token)+1
        vali_y_seq.append(['\t']+seq+['\n'])

In [19]:
with open(tokenfile,'wb') as file:
    pickle.dump(token,file)
    
num_tokens=len(token)
batch_size=1
seq_len=49+2 #49+'\t'+'\n'
vec_len=25088
print("Num of train:",len(train_x_file))
print("Num of vali:",len(vali_x_file))
print("Num of token:",num_tokens)
print("Len of vector:",vec_len)
c = list(zip(train_x_file, train_y_seq))

random.shuffle(c)

train_x_file, train_y_seq = zip(*c)

Num of train: 82783
Num of vali: 5000
Num of token: 11857
Len of vector: 25088


In [7]:
def vec_reader(path):
    with open(path,'r') as f:
        line=f.readline()
        vector=np.asarray( list(map(float,line.split(",")[1].split(" "))) )
    return vector

In [15]:
vggmodel = VGG16(weights='imagenet', include_top=False)
def vggextract(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return np.reshape(vggmodel.predict(x),(25088))

In [17]:
a=vggextract('./file/000001.jpg')

In [9]:
def dataGenerator(x,y):
    while(1):
        decoder_input_data=[]
        decoder_target_data=[]
        encoder_input_data=[]
        for index in range(len(x)):
            for t,word in enumerate(y[index]):
                if t>0:
                    did=np.zeros(
                        (seq_len),dtype='float32'
                    )
                    dtd=np.zeros(
                        (1),dtype='float32'
                    )
                    eid=vggextract(x[index])
                    #eid=vec_reader(x[index]+'.txt')
                    #eid=np.reshape(eid,(1,vec_len))
                    encoder_input_data.append(eid)
                    for tt in range(t):
                        did[tt]=float(token[y[index][tt]])
                    dtd[0]=float(token[y[index][t]])
                    decoder_input_data.append(did)
                    decoder_target_data.append(dtd)
                    if len(encoder_input_data)==batch_size:
                        yield ([np.array(encoder_input_data),np.array(decoder_input_data)],np.array(decoder_target_data))
                        decoder_input_data=[]
                        decoder_target_data=[]
                        encoder_input_data=[]

In [10]:
latent_dim=256
encode_input=Input(shape=(vec_len,))
encode_dense=Dense(latent_dim,activation='relu')(encode_input)
#encode_dense=Reshape((1,latent_dim))(encode_dense)
encode_dense=RepeatVector(1)(encode_dense)
decode_input=Input(shape=(seq_len,))
decode_emb=Embedding(num_tokens+1,latent_dim,input_length=(seq_len))(decode_input)
decoder_vec=concatenate([decode_emb,encode_dense],axis=1)
decoder=LSTM(latent_dim)(decoder_vec)
decoder=Dense(num_tokens,activation='softmax')(decoder)
model=Model([encode_input,decode_input],decoder)

In [11]:
def ppx(y_true, y_pred):
     loss = K.sparse_categorical_crossentropy(y_true, y_pred)
     perplexity = K.cast(K.pow(math.e, K.mean(loss, axis=-1)), K.floatx())
     return perplexity

In [12]:
model.compile(loss=ppx, optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit_generator(
    dataGenerator(train_x_file,train_y_seq),
    #validation_data=dataGenerator(vali_x_file,vali_y_seq),
    steps_per_epoch=((len(train_x_file))//batch_size),
    #steps_per_epoch=10000,
    #validation_steps=1,
    epochs=200
    #callbacks=[cbes]
)

Epoch 1/200
82783/82783 [==============================] - 3557s 43ms/step - loss: 525189.3511 - acc: 0.0000e+00
Epoch 2/200
82783/82783 [==============================] - 3557s 43ms/step - loss: 496203.8144 - acc: 0.0000e+00
Epoch 3/200
82783/82783 [==============================] - 3556s 43ms/step - loss: 479043.2914 - acc: 0.0000e+00
Epoch 4/200
82783/82783 [==============================] - 3558s 43ms/step - loss: 521280.9565 - acc: 0.0000e+00
Epoch 5/200
82783/82783 [==============================] - 3557s 43ms/step - loss: 537205.2470 - acc: 0.0000e+00
Epoch 6/200
82783/82783 [==============================] - 3558s 43ms/step - loss: 546644.9061 - acc: 0.0000e+00
Epoch 7/200
82783/82783 [==============================] - 3557s 43ms/step - loss: 575411.7190 - acc: 0.0000e+00
Epoch 8/200
82783/82783 [==============================] - 3557s 43ms/step - loss: 537917.1786 - acc: 0.0000e+00
Epoch 9/200
82783/82783 [==============================] - 3573s 43ms/step - loss: 531968.5134 -

In [ ]:
if os.path.exists(detokenfile):
    with open(detokenfile,'rb') as file:
        detoken=pickle.load(file)
else:
    detoken=dict((i,char) for char,i in token.items())
    with open(detokenfile,'wb') as file:
        pickle.dump(detoken,file)

def decode(code,seq,seq_index,sentence):
    decode_token_index=np.argmax(model.predict([code,seq])[0])
    seq[0][seq_index]=decode_token_index
    sampled_char=detoken[decode_token_index]
    sentence+=(" "+sampled_char)
    if seq_index+1==seq_len or sampled_char='\n':
        return sentence
    else:
        return decode(code,seq,seq_index+1,sentence)
    
indices=random.sample(range(len(vali_x_file)),32)
for index in indices:
    code=vggextract(vali_x_file[index])
    code=np.reshape(code,(1,code.shape[0]))
    did=np.zeros(
        (seq_len),dtype='float32'
    )
    did=np.reshape(did,(1,did.shape[0]))
    did[0][0]=float(token['\t'])
    sentence=decode(code,did,1,"")
    print("-")
    print("Input")
    plt.imshow(np.asarray(Image.open(vali_x_file[index])))
    plt.show()
    print("Decoded sentence:",sentence)
    print("should be:",vali_y_seq_index)